<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gluonnlp tqdm

In [4]:
pip install pytorch_transformers

In [3]:
pip install mxnet

In [1]:
pip install transformers

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [ ]:
# 모델과 tokenizer 준비

model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

# 동일 문장 여부 파악

In [ ]:
# 임의의 문장 예측 (네이버 기사)

# 이어지는 문장
prompt1 = '정부는 약사 등의 '
next_sentence1 = '환자 발생으로 약국 영업이 중단될 경우 해당 지역의 보건 서비스에 심각한 영향이 우려돼 이번 지침을 마련했다고 설명했다.'

# 이어지지 않는 문장
prompt2 = '튜브 겉면에도 복합 재질을 의미하는 '
next_sentence2 = '연기를 할 수 없어 이같은 방법을 선택한 것으로 보인다.'

In [ ]:
def same_sentence(prompt, next):
  encoding = tokenizer(prompt, next, return_tensors='tf')
  logits = model(encoding['input_ids'], token_type_ids = encoding['token_type_ids'])[0]
  softmax = Softmax()
  probs = softmax(logits)
  res = tf.math.argmax(probs, axis=-1).numpy()

  return res

In [ ]:
res1 = same_sentence(prompt1, next_sentence1)

if res1 == 0:
  print('두 문장은 이어지는 문장입니다.')
else:
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지는 문장입니다.


In [ ]:
res2 = same_sentence(prompt2, next_sentence2)

if res2 == 0:
  print('두 문장은 이어지는 문장입니다.')
else: 
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지지 않는 문장입니다.


# 데이터셋 준비

In [ ]:
import os

root = '/content/drive/MyDrive/news_class9x13000/'
categories = os.listdir(root)

dataset = []

for category in categories:
  files = os.listdir(root + category)
  for i, f in enumerate(files):
    fname = root + category + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()
  

In [20]:
len(dataset)

117010

In [39]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '▶': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': '',
    '‘': '',
    '■': '',
    '○': ''

})

In [22]:
process_dataset = []

for sentences in dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split('\n')
  del sentences[0]
  process_dataset.extend(sentences)

process_dataset

['프로크루스테스의 침대라는 그리스신화에 나오는 이야기가 있다.',
 '이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대에 눕혀놓고 나그네의 키가 침대보다 길면 발목을 자르고 침대보다 짧으면 다리를 강제로 늘린다는 내용을 담고 있다.',
 '결국 침대에 사람을 맞춘다는 얘기다.',
 '올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.',
 '장애인활동보조서비스는 장애로 인해 일상생활의 어려움을 겪는 사람들의 원활한 일상생활과 사회활동을 돕는 인력파견 장애인복지서비스다.',
 '2007년부터 시작한 이 서비스 제도는 전국의 중증장애인들이 국가를 상대로 치열한 투쟁을 통해 획득한 결과물이다.',
 '중증장애인들은 이러한 결과물을 얻기 위해 국가인권위원회를 점거하기도 했고 40여명의 장애인 당사자들이 단식을 하기도 했고 삭발을 통해 결의를 보였으며 40여명의 중증장애인들이 장장 4시간에 걸쳐 한강대교를 온몸으로 구르고 사력을 다해 기었다.',
 '그렇게 치열한 사투 끝에 만들어 낸 제도다.',
 '그러나 보건복지부는 이 제도의 첫 시행부터 중증장애인들에게 만족할 만한 활도보조서비스(활보서비스)를 설계하지 못했다.',
 '아니 애초부터 만들지 않았다는 게 맞을 것이다.',
 '우리는 이 활보서비스를 중증장애인의 당연한 권리로서 누구든 필요한 시간만큼 아무런 조건 없이 무상으로 제공해야 한다고 요구했지만 정부(보건복지부)는 2007년 첫 시행부터 대상을 제한하고 시간을 제한하고 이용하는 장애인들에게 자부담을 하도록 강요했다.',
 '그런데 올해부터 한술 더 떠서 활동보조서비스 시행규칙을 악질적으로 개정했다.',
 '가장 크게 문제가 되고 있는 것은 활보서비스를 계속 받으려면 장애등급 재심사를 받아야 하는 것이고 두 번째는 활보 이용자 자부담을 인상한 것이다.',
 '비장애인에 비해 장애인의 실업률이 높고 이로 인해 가족들에게 손 벌려야 하는데 이것 자체로 그들이 받는 수치감은 엄청 크다.',
 '이런 이유

In [24]:
len(process_dataset)

4083435

In [23]:
with open(
    os.path.join(root, 'delete_title'), mode = 'w', encoding = 'utf-8'
) as f:
  for data in process_dataset:
    f.write(data)

print('save the preprocessing data (del title version) file')

save the preprocessing data (del title version) file


## 데이터 만들기

    corpus = [prompt, next, isNext]

    for index in range(process_dataset):
      if len(corpus) % 2 == 0: 
        이어지는 문장
        corpus.append([process_dataset[index - 1], process_dataset[index], True])
      else: 
        이어지지 않는 문장
        randomIndex = randint(0, len(corpus)-1)
        corpus.append([process_dataset[randomIndex], process_dataset[index], False])





In [31]:
from random import randint

corpus = []

for index in range(0, len(process_dataset)-1):
  if len(corpus) % 2 == 0:
    # 이어지는 문장
    prompt = process_dataset[index]
    next = process_dataset[index + 1]
    if same_sentence(prompt, next) == 0:
      corpus.append([prompt, next, True])
    else: 
      corpus.append([prompt, next, False])
  else:
    # 이어지지 않는 문장
    random_index = randint(0, len(process_dataset)-1)
    corpus.append([process_dataset[index], process_dataset[random_index], False])

len(corpus)

KeyboardInterrupt: ignored

In [38]:
import pandas as pd

corpus_dataset = pd.DataFrame(corpus, columns = ['prompt', 'next', 'isNext'])

corpus_dataset

,prompt,next,isNext
0,프로크루스테스의 침대라는 그리스신화에 나오는 이야기가 있다.,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...,True
1,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...,하지만 이한구 원내대표의 색깔론 제기가 오히려 사회 불안을 조장한다는 비판이 나온다.,False
2,결국 침대에 사람을 맞춘다는 얘기다.,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.,False
3,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.,"더불어 내가 즐겨 입는 옷들만 남겨두니, 오늘은 뭐 입지 라는 고민을 하지 않게 되...",False
4,장애인활동보조서비스는 장애로 인해 일상생활의 어려움을 겪는 사람들의 원활한 일상생활...,2007년부터 시작한 이 서비스 제도는 전국의 중증장애인들이 국가를 상대로 치열한 ...,True
...,...,...,...
15827,담임교사가 학기초부터 꾸준히 여러 경로로 아이들을 파악하고 있고 학교 수준에서 자율...,북측 감독과 심판이었다.,False
15828,휴대폰 수거는 기본이고 책상 줄까지 일일이 규정하고 있다.,영어 듣기 평가도 딱 1번만 들려주란다.,False
15829,영어 듣기 평가도 딱 1번만 들려주란다.,"육체적, 감정적 욕구를 충족시키기 위해 사방을 둘러보며, 그 과정에서 고통스러운 실...",False
15830,시험실 설치,"○ 책걸상 배열 : 수험번호 순, 5열×7명(35명 초과 학급은 5열×8~9명)",True


In [41]:
corpus_dataset.isNext.value_counts()

False    8857
True     6975
Name: isNext, dtype: int64

# Fine tuning

1. 데이터셋준비 (완료)
2. 데이터 정제
3. 데이터 토큰화
4. 데이터 패딩 및 attention mask
5. train, val, test dataset
6. train, val, test를 torch tensor로 변환
7. TensorDataset으로 train input, mask, label 묶기
8. 모델 device를 `cuda:0`으로 설정
 (`model.to(device)` )
9. BertForSequenceClassification.from_pretrained => pretrained 된 bert 불러오기

  [간단한 예제](https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/)

10. 옵티마이저 설정
11. 스케줄러 설정
12. 랜덤 시드 고정
13. 모델 그래디언트 초기화
14. Epoch만큼 훈련 반복

참고 : [BERT 모델의 장점과 Fine-Tuning 흐름 정리](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=antler07&logNo=222066071573)

In [50]:
import torch
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))